In [1]:
import pandas as pd 
import numpy as np

import ast
import nest_asyncio         # nest_asyncio는 파이썬의 비동기 코드 실행을 도와주는 패키지
nest_asyncio.apply()        # Jupyter 노트북과 같이 이벤트 루프가 이미 실행 중인 환경에서 비동기 코드를 여러 번 실행할 수 있도록 설정하는 데 사용
# 이거 안하면 무조건 에러남 

from datasets import Dataset 
from ragas import evaluate

In [2]:
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,      # component-wise 평가 방법
    answer_correctness,
    answer_similarity       # end-to-end 평가 방법
    
)

metrics = [
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
    answer_correctness,
    answer_similarity
]


In [3]:
def set_data(file):
    qa_data = pd.read_excel(file)
    for col in ['contexts']:
        qa_data[col] = qa_data[col].apply(ast.literal_eval)
        
    dataset = Dataset.from_pandas(qa_data)
    
    return qa_data, dataset

In [4]:
file = 'experiment_dataset\QA\setting0_retriever0.xlsx'

qa_data, dataset = set_data(file)

<>:1: SyntaxWarning: invalid escape sequence '\Q'
<>:1: SyntaxWarning: invalid escape sequence '\Q'
C:\Users\kim_h\AppData\Local\Temp\ipykernel_30492\822981208.py:1: SyntaxWarning: invalid escape sequence '\Q'
  file = 'experiment_dataset\QA\setting0_retriever0.xlsx'


In [5]:
qa_data

,question,ground_truth,answer,contexts
0,2020년 1월부터 8월까지 인도가 한국을 대상으로 개시한 반덤핑 조사는 몇 건인가요?,2020년 1월부터 8월까지 인도가 한국을 대상으로 개시한 반덤핑 조사는 3건입니다.,2020년 1월부터 8월까지 인도가 한국을 대상으로 개시한 반덤핑 조사는 총 3건이...,[]
1,A사는 인도에서 어떤 제품을 생산하고 있나요?,A사는 인도에서 스마트폰을 생산하고 있습니다.,죄송합니다. 제가 현재 A사가 인도에서 어떤 제품을 생산하는지에 대한 정보를 가지고...,[]
2,인도의 국경 간 무역 용이성 수준은 얼마인가요?,인도의 국경 간 무역 용이성 수준은 77.46입니다.,인도의 국경 간 무역 용이성 수준은 세계은행의 '무역 용이성 지수'를 통해 측정됩니...,[]
3,한-인도 CEPA 적용을 위한 원산지증명서 서식은 무엇에 따라 달라질 수 있나요?,한-인도 CEPA 적용을 위한 원산지증명서 서식은 한국과 인도의 양국에서 사용하는 ...,한-인도 CEPA(Comprehensive Economic Partnership A...,[]
4,인도의 무역구제규정 개정의 주요 내용은 무엇인가요?,"인도의 무역구제규정 개정의 주요 내용은 WTO법과의 합치성을 제고하고, 특정 용어에...",죄송합니다. 현재 제공된 정보에는 인도의 무역구제규정 개정 내용에 대한 구체적인 정...,[]
5,BIS 인증을 위해 샘플을 인도에 발송할 때 주의해야 할 사항은 무엇인가요?,BIS 인증을 위해 샘플을 인도에 발송할 때는 글로벌 특송업체를 이용하는 것이 좋으...,BIS(Bureau of Indian Standards) 인증을 위해 샘플을 인도에...,[]
6,인도에 중고 설비를 수출할 때 한-인도 CEPA를 활용하기 위해 필요한 서류는 무엇...,중고 설비를 수출할 때 한-인도 CEPA를 활용하기 위해서는 수출국의 공인 감정평가...,한-인도 CEPA(Comprehensive Economic Partnership A...,[]
7,인도 무역구제제도 운영의 최근 변화는 무엇인가요?,최근 인도 무역구제제도 운영에서는 법규 제·개정 및 매뉴얼 작업을 통해 조사 과정상...,죄송합니다. 제가 현재 인도 무역구제제도 운영의 최근 변화에 대한 정보를 가지고 있...,[]
8,인도에서 식품 수입 시 필요한 필수 서류는 무엇인가요?,인도에서 식품 수입 시 필요한 필수 서류는 인도 상공부 무역국(DGFT)에서 발행한...,인도에서 식품 수입 시 필요한 필수 서류는 다음과 같습니다:\n1. 수입 허가서\n...,[]
9,FSSAI 라이선스 발급 절차는 몇 단계로 이루어져 있나요?,FSSAI 라이선스 발급 절차는 3단계로 이루어져 있습니다.,FSSAI(식품 안전 표준 및 규정 기관) 라이선스 발급은 다음과 같은 단계로 이루...,[]


In [6]:
len(qa_data)

11

In [8]:
from ragas import evaluate

metrics_gen = [faithfulness,answer_relevancy]

result = evaluate(
    dataset,
    metrics=metrics_gen,
    raise_exceptions=False  # Add this line
)

ValueError: Dataset feature "contexts" should be of type Sequence[string], got <class 'datasets.features.features.Sequence'>

In [8]:
df1 = pd.DataFrame(list(result.items()), columns=['Metric', 'setting1'])


df1

,Metric,setting1
0,faithfulness,0.333333
1,answer_relevancy,0.589395
2,answer_correctness,0.352464
3,context_recall,0.200000
4,context_precision,0.691667


In [9]:
df1.set_index('Metric', inplace=True)
df1 = df1.transpose().reset_index()

df1

Metric,index,faithfulness,answer_relevancy,answer_correctness,context_recall,context_precision
0,setting1,0.333333,0.589395,0.352464,0.2,0.691667


In [9]:
import os 
import glob 

In [10]:
# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting1_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

In [11]:
file_list

['./experiment_dataset/QA\\setting1_retriever1_003.xlsx',
 './experiment_dataset/QA\\setting1_retriever1_005.xlsx',
 './experiment_dataset/QA\\setting1_retriever1_007.xlsx',
 './experiment_dataset/QA\\setting1_retriever1_010.xlsx']

In [12]:
file_list[0].split('\\')[-1].split('.')[0]

'setting1_retriever1_003'

In [13]:
from ragas.run_config import RunConfig
# RAGAS Evaluation TimeoutError를 해결하기 위해 

In [19]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings

# llm = ChatOllama(model="gemma2")
# embeddings = OllamaEmbeddings(model="gemma2")


llm = ChatOpenAI(model= 'gpt-4o')
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")    


In [20]:
def merge_result(file_list):
    for i, file in enumerate(file_list):
        set_name = file.split('\\')[-1].split('.')[0]
        print(set_name)
        _, dataset = set_data(file)
        # raise_exceptions=False로 하면 평가시 nan값이 발생했을 때, 평가 종합 계산시 제외한다.
        result = evaluate(dataset, 
                          metrics=metrics, 
                          raise_exceptions=False, 
                          run_config=RunConfig(timeout=120, max_retries=5, max_wait=120, max_workers=32),
                          llm = llm,
                          embeddings=embeddings
                          )
        
        if i == 0:
            df = pd.DataFrame(list(result.items()) + [('mean', np.mean(list(result.values())))], columns=['Metric', set_name])

        else:
            df_temp = pd.DataFrame(list(result.items()) + [('mean', np.mean(list(result.values())))], columns=['Metric', set_name])
            df = pd.merge(df, df_temp, on='Metric')
        
        print(df)
    
    
    df.set_index('Metric', inplace=True)
    df = df.transpose().reset_index()
    
    return df
           

In [21]:
import nest_asyncio     
nest_asyncio.apply()  

In [22]:
df = merge_result(file_list)

setting1_retriever1_003


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting1_retriever1_003
0        faithfulness                 0.747147
1    answer_relevancy                 0.767905
2      context_recall                 0.636364
3   context_precision                 0.636364
4  answer_correctness                 0.498669
5   answer_similarity                 0.943480
6                mean                 0.704988
setting1_retriever1_005


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting1_retriever1_003  setting1_retriever1_005
0        faithfulness                 0.747147                 0.838312
1    answer_relevancy                 0.767905                 0.834782
2      context_recall                 0.636364                 0.818182
3   context_precision                 0.636364                 0.661364
4  answer_correctness                 0.498669                 0.530282
5   answer_similarity                 0.943480                 0.940168
6                mean                 0.704988                 0.770515
setting1_retriever1_007


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting1_retriever1_003  setting1_retriever1_005  \
0        faithfulness                 0.747147                 0.838312   
1    answer_relevancy                 0.767905                 0.834782   
2      context_recall                 0.636364                 0.818182   
3   context_precision                 0.636364                 0.661364   
4  answer_correctness                 0.498669                 0.530282   
5   answer_similarity                 0.943480                 0.940168   
6                mean                 0.704988                 0.770515   

   setting1_retriever1_007  
0                 0.806006  
1                 0.752260  
2                 0.818182  
3                 0.660714  
4                 0.484692  
5                 0.948456  
6                 0.745052  
setting1_retriever1_010


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\metrics\_context_precision.py", line 161, in _ascore
    results = await self.llm.generate(
              ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 111, in __call__
    do = await self.iter(retry_state=retry_state)
         ^^^^^^^^^^^^^^

               Metric  setting1_retriever1_003  setting1_retriever1_005  \
0        faithfulness                 0.747147                 0.838312   
1    answer_relevancy                 0.767905                 0.834782   
2      context_recall                 0.636364                 0.818182   
3   context_precision                 0.636364                 0.661364   
4  answer_correctness                 0.498669                 0.530282   
5   answer_similarity                 0.943480                 0.940168   
6                mean                 0.704988                 0.770515   

   setting1_retriever1_007  setting1_retriever1_010  
0                 0.806006                 0.896620  
1                 0.752260                 0.809962  
2                 0.818182                 0.818182  
3                 0.660714                 0.631786  
4                 0.484692                 0.488367  
5                 0.948456                 0.944312  
6                 0.7

In [23]:
df

Metric,index,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness,answer_similarity,mean
0,setting1_retriever1_003,0.747147,0.767905,0.636364,0.636364,0.498669,0.943480,0.704988
1,setting1_retriever1_005,0.838312,0.834782,0.818182,0.661364,0.530282,0.940168,0.770515
2,setting1_retriever1_007,0.806006,0.752260,0.818182,0.660714,0.484692,0.948456,0.745052
3,setting1_retriever1_010,0.896620,0.809962,0.818182,0.631786,0.488367,0.944312,0.764871


In [24]:
df.to_excel('./experiment_dataset/QA_results/setting1_retriever1.xlsx', index=False)

In [25]:
# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting2_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

print(file_list)

['./experiment_dataset/QA\\setting2_retriever2_003.xlsx', './experiment_dataset/QA\\setting2_retriever2_005.xlsx', './experiment_dataset/QA\\setting2_retriever2_007.xlsx', './experiment_dataset/QA\\setting2_retriever2_010.xlsx']


In [26]:
import nest_asyncio     
nest_asyncio.apply()  

In [27]:
df = merge_result(file_list)

setting2_retriever2_003


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

Runner in Executor raised an exception
Traceback (most recent call last):
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\asyncio\tasks.py", line 520, in wait_for
    return await fut
           ^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\metrics\_answer_correctness.py", line 250, in _ascore
    is_statement_present = await self.llm.generate(
                           ^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\ragas\llms\base.py", line 93, in generate
    return await agenerate_text_with_retry(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 189, in async_wrapped
    return await copy(fn, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\kim_h\anaconda3\envs\AUTORAG\Lib\site-packages\tenacity\asyncio\__init__.py", line 111, in __call__
    do = await self.iter(retry_state=retry_sta

               Metric  setting2_retriever2_003
0        faithfulness                 0.840152
1    answer_relevancy                 0.949337
2      context_recall                 0.727273
3   context_precision                 0.636364
4  answer_correctness                 0.501548
5   answer_similarity                 0.938901
6                mean                 0.765596
setting2_retriever2_005


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

...

               Metric  setting2_retriever2_003  setting2_retriever2_005
0        faithfulness                 0.840152                 0.845455
1    answer_relevancy                 0.949337                 0.832989
2      context_recall                 0.727273                 0.818182
3   context_precision                 0.636364                 0.727273
4  answer_correctness                 0.501548                 0.527678
5   answer_similarity                 0.938901                 0.943289
6                mean                 0.765596                 0.782477
setting2_retriever2_007


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting2_retriever2_003  setting2_retriever2_005  \
0        faithfulness                 0.840152                 0.845455   
1    answer_relevancy                 0.949337                 0.832989   
2      context_recall                 0.727273                 0.818182   
3   context_precision                 0.636364                 0.727273   
4  answer_correctness                 0.501548                 0.527678   
5   answer_similarity                 0.938901                 0.943289   
6                mean                 0.765596                 0.782477   

   setting2_retriever2_007  
0                 0.790548  
1                 0.912493  
2                 0.818182  
3                 0.757576  
4                 0.484046  
5                 0.937619  
6                 0.783411  
setting2_retriever2_010


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting2_retriever2_003  setting2_retriever2_005  \
0        faithfulness                 0.840152                 0.845455   
1    answer_relevancy                 0.949337                 0.832989   
2      context_recall                 0.727273                 0.818182   
3   context_precision                 0.636364                 0.727273   
4  answer_correctness                 0.501548                 0.527678   
5   answer_similarity                 0.938901                 0.943289   
6                mean                 0.765596                 0.782477   

   setting2_retriever2_007  setting2_retriever2_010  
0                 0.790548                 0.831457  
1                 0.912493                 0.741315  
2                 0.818182                 0.818182  
3                 0.757576                 0.695455  
4                 0.484046                 0.485876  
5                 0.937619                 0.946969  
6                 0.7

In [28]:

df

Metric,index,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness,answer_similarity,mean
0,setting2_retriever2_003,0.840152,0.949337,0.727273,0.636364,0.501548,0.938901,0.765596
1,setting2_retriever2_005,0.845455,0.832989,0.818182,0.727273,0.527678,0.943289,0.782477
2,setting2_retriever2_007,0.790548,0.912493,0.818182,0.757576,0.484046,0.937619,0.783411
3,setting2_retriever2_010,0.831457,0.741315,0.818182,0.695455,0.485876,0.946969,0.753209


In [29]:
df.to_excel('./experiment_dataset/QA_results/setting2_retriever2.xlsx', index=False)

In [30]:
import nest_asyncio     
nest_asyncio.apply()  

# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting3_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

print(file_list)

df = merge_result(file_list)

df.to_excel('./experiment_dataset/QA_results/setting3_retriever3.xlsx', index=False)

['./experiment_dataset/QA\\setting3_retriever3_003_001.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_002.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_003.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_004.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_005.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_006.xlsx', './experiment_dataset/QA\\setting3_retriever3_003_007.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_001.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_002.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_003.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_004.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_005.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_006.xlsx', './experiment_dataset/QA\\setting3_retriever3_005_007.xlsx', './experiment_dataset/QA\\setting3_retriever3_007_001.xlsx', './experiment_dataset/QA\\setting3_retriever3_007_002.xlsx', './experiment_dataset/Q

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting3_retriever3_003_001
0        faithfulness                     0.750541
1    answer_relevancy                     0.771468
2      context_recall                     0.545455
3   context_precision                     0.636364
4  answer_correctness                     0.357326
5   answer_similarity                     0.927871
6                mean                     0.664837
setting3_retriever3_003_002


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting3_retriever3_003_001  \
0        faithfulness                     0.750541   
1    answer_relevancy                     0.771468   
2      context_recall                     0.545455   
3   context_precision                     0.636364   
4  answer_correctness                     0.357326   
5   answer_similarity                     0.927871   
6                mean                     0.664837   

   setting3_retriever3_003_002  
0                     0.742965  
1                     0.776203  
2                     0.545455  
3                     0.636364  
4                     0.416678  
5                     0.932985  
6                     0.675108  
setting3_retriever3_003_003


...

               Metric  setting3_retriever3_003_001  \
0        faithfulness                     0.750541   
1    answer_relevancy                     0.771468   
2      context_recall                     0.545455   
3   context_precision                     0.636364   
4  answer_correctness                     0.357326   
5   answer_similarity                     0.927871   
6                mean                     0.664837   

   setting3_retriever3_003_002  setting3_retriever3_003_003  \
0                     0.742965                     0.693182   
1                     0.776203                     0.778978   
2                     0.545455                     0.545455   
3                     0.636364                     0.636364   
4                     0.416678                     0.353778   
5                     0.932985                     0.928692   
6                     0.675108                     0.656075   

   setting3_retriever3_003_004  setting3_retriever3_003_005  \

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting3_retriever3_003_001  \
0        faithfulness                     0.750541   
1    answer_relevancy                     0.771468   
2      context_recall                     0.545455   
3   context_precision                     0.636364   
4  answer_correctness                     0.357326   
5   answer_similarity                     0.927871   
6                mean                     0.664837   

   setting3_retriever3_003_002  setting3_retriever3_003_003  \
0                     0.742965                     0.693182   
1                     0.776203                     0.778978   
2                     0.545455                     0.545455   
3                     0.636364                     0.636364   
4                     0.416678                     0.353778   
5                     0.932985                     0.928692   
6                     0.675108                     0.656075   

   setting3_retriever3_003_004  setting3_retriever3_003_005  \

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting3_retriever3_003_001  \
0        faithfulness                     0.750541   
1    answer_relevancy                     0.771468   
2      context_recall                     0.545455   
3   context_precision                     0.636364   
4  answer_correctness                     0.357326   
5   answer_similarity                     0.927871   
6                mean                     0.664837   

   setting3_retriever3_003_002  setting3_retriever3_003_003  \
0                     0.742965                     0.693182   
1                     0.776203                     0.778978   
2                     0.545455                     0.545455   
3                     0.636364                     0.636364   
4                     0.416678                     0.353778   
5                     0.932985                     0.928692   
6                     0.675108                     0.656075   

   setting3_retriever3_003_004  setting3_retriever3_003_005  \

In [31]:
import nest_asyncio     
nest_asyncio.apply()  

# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting4_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

print(file_list)

df = merge_result(file_list)

df.to_excel('./experiment_dataset/QA_results/setting4_our_retriever.xlsx', index=False)

['./experiment_dataset/QA\\setting4_our_retriever_003_001.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_002.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_003.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_004.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_005.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_006.xlsx', './experiment_dataset/QA\\setting4_our_retriever_003_007.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_001.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_002.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_003.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_004.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_005.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_006.xlsx', './experiment_dataset/QA\\setting4_our_retriever_005_007.xlsx', './experiment_dataset/QA\\setting4_our_retriever_007_001.xlsx', './experiment_dataset/QA\\setting4_our_

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting4_our_retriever_003_001
0        faithfulness                        0.818182
1    answer_relevancy                        0.771211
2      context_recall                        0.636364
3   context_precision                        0.636364
4  answer_correctness                        0.509174
5   answer_similarity                        0.943704
6                mean                        0.719166
setting4_our_retriever_003_002


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting4_our_retriever_003_001  \
0        faithfulness                        0.818182   
1    answer_relevancy                        0.771211   
2      context_recall                        0.636364   
3   context_precision                        0.636364   
4  answer_correctness                        0.509174   
5   answer_similarity                        0.943704   
6                mean                        0.719166   

   setting4_our_retriever_003_002  
0                        0.798485  
1                        0.772628  
2                        0.636364  
3                        0.636364  
4                        0.503659  
5                        0.941787  
6                        0.714881  
setting4_our_retriever_003_003


...

               Metric  setting4_our_retriever_003_001  \
0        faithfulness                        0.818182   
1    answer_relevancy                        0.771211   
2      context_recall                        0.636364   
3   context_precision                        0.636364   
4  answer_correctness                        0.509174   
5   answer_similarity                        0.943704   
6                mean                        0.719166   

   setting4_our_retriever_003_002  setting4_our_retriever_003_003  \
0                        0.798485                        0.767857   
1                        0.772628                        0.777435   
2                        0.636364                        0.636364   
3                        0.636364                        0.636364   
4                        0.503659                        0.536740   
5                        0.941787                        0.941748   
6                        0.714881                        0.7

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting4_our_retriever_003_001  \
0        faithfulness                        0.818182   
1    answer_relevancy                        0.771211   
2      context_recall                        0.636364   
3   context_precision                        0.636364   
4  answer_correctness                        0.509174   
5   answer_similarity                        0.943704   
6                mean                        0.719166   

   setting4_our_retriever_003_002  setting4_our_retriever_003_003  \
0                        0.798485                        0.767857   
1                        0.772628                        0.777435   
2                        0.636364                        0.636364   
3                        0.636364                        0.636364   
4                        0.503659                        0.536740   
5                        0.941787                        0.941748   
6                        0.714881                        0.7

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting4_our_retriever_003_001  \
0        faithfulness                        0.818182   
1    answer_relevancy                        0.771211   
2      context_recall                        0.636364   
3   context_precision                        0.636364   
4  answer_correctness                        0.509174   
5   answer_similarity                        0.943704   
6                mean                        0.719166   

   setting4_our_retriever_003_002  setting4_our_retriever_003_003  \
0                        0.798485                        0.767857   
1                        0.772628                        0.777435   
2                        0.636364                        0.636364   
3                        0.636364                        0.636364   
4                        0.503659                        0.536740   
5                        0.941787                        0.941748   
6                        0.714881                        0.7

In [40]:
import nest_asyncio     
nest_asyncio.apply()  

# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting5_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

print(file_list)

df = merge_result(file_list)

df.to_excel('./experiment_dataset/QA_results/setting5_mq_retriever.xlsx', index=False)



['./experiment_dataset/QA\\setting5_mq_retriever.xlsx', './experiment_dataset/QA\\setting5_mq_retriever_000.xlsx', './experiment_dataset/QA\\setting5_mq_retriever_001.xlsx', './experiment_dataset/QA\\setting5_mq_retriever_002.xlsx', './experiment_dataset/QA\\setting5_mq_retriever_003.xlsx']
setting5_mq_retriever


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting5_mq_retriever
0        faithfulness               0.897619
1    answer_relevancy               0.817189
2      context_recall               0.818182
3   context_precision               0.562987
4  answer_correctness               0.538979
5   answer_similarity               0.945551
6                mean               0.763418
setting5_mq_retriever_000


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting5_mq_retriever  setting5_mq_retriever_000
0        faithfulness               0.897619                   0.956061
1    answer_relevancy               0.817189                   0.813061
2      context_recall               0.818182                   0.727273
3   context_precision               0.562987                   0.573485
4  answer_correctness               0.538979                   0.503129
5   answer_similarity               0.945551                   0.938030
6                mean               0.763418                   0.751840
setting5_mq_retriever_001


...

               Metric  setting5_mq_retriever  setting5_mq_retriever_000  \
0        faithfulness               0.897619                   0.956061   
1    answer_relevancy               0.817189                   0.813061   
2      context_recall               0.818182                   0.727273   
3   context_precision               0.562987                   0.573485   
4  answer_correctness               0.538979                   0.503129   
5   answer_similarity               0.945551                   0.938030   
6                mean               0.763418                   0.751840   

   setting5_mq_retriever_001  
0                   0.809877  
1                   0.717175  
2                   0.818182  
3                   0.580303  
4                   0.462504  
5                   0.945321  
6                   0.722227  
setting5_mq_retriever_002


Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting5_mq_retriever  setting5_mq_retriever_000  \
0        faithfulness               0.897619                   0.956061   
1    answer_relevancy               0.817189                   0.813061   
2      context_recall               0.818182                   0.727273   
3   context_precision               0.562987                   0.573485   
4  answer_correctness               0.538979                   0.503129   
5   answer_similarity               0.945551                   0.938030   
6                mean               0.763418                   0.751840   

   setting5_mq_retriever_001  setting5_mq_retriever_002  
0                   0.809877                   0.855051  
1                   0.717175                   0.831058  
2                   0.818182                   0.818182  
3                   0.580303                   0.575758  
4                   0.462504                   0.564834  
5                   0.945321                   0.94

Evaluating:   0%|          | 0/66 [00:00<?, ?it/s]

               Metric  setting5_mq_retriever  setting5_mq_retriever_000  \
0        faithfulness               0.897619                   0.956061   
1    answer_relevancy               0.817189                   0.813061   
2      context_recall               0.818182                   0.727273   
3   context_precision               0.562987                   0.573485   
4  answer_correctness               0.538979                   0.503129   
5   answer_similarity               0.945551                   0.938030   
6                mean               0.763418                   0.751840   

   setting5_mq_retriever_001  setting5_mq_retriever_002  \
0                   0.809877                   0.855051   
1                   0.717175                   0.831058   
2                   0.818182                   0.818182   
3                   0.580303                   0.575758   
4                   0.462504                   0.564834   
5                   0.945321                 

In [41]:
df.head()

Metric,index,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness,answer_similarity,mean
0,setting5_mq_retriever,0.897619,0.817189,0.818182,0.562987,0.538979,0.945551,0.763418
1,setting5_mq_retriever_000,0.956061,0.813061,0.727273,0.573485,0.503129,0.938030,0.751840
2,setting5_mq_retriever_001,0.809877,0.717175,0.818182,0.580303,0.462504,0.945321,0.722227
3,setting5_mq_retriever_002,0.855051,0.831058,0.818182,0.575758,0.564834,0.944891,0.764962
4,setting5_mq_retriever_003,0.878896,0.827737,0.818182,0.558442,0.557697,0.947701,0.764776


In [37]:
import nest_asyncio     
nest_asyncio.apply()  

# 파일 경로 패턴 설정
file_pattern = './experiment_dataset/QA/setting6_*.xlsx'

# 파일 리스트 가져오기
file_list = glob.glob(file_pattern)

print(file_list)

df = merge_result(file_list)

df.to_excel('./experiment_dataset/QA_results/setting6_sq_retriever.xlsx', index=False)


['./experiment_dataset/QA\\setting6_sq_retriever.xlsx']
setting6_sq_retriever


Evaluating:   0%|          | 0/60 [00:00<?, ?it/s]

               Metric  setting6_sq_retriever
0        faithfulness               0.258532
1    answer_relevancy               0.553217
2      context_recall               0.100000
3   context_precision               0.225000
4  answer_correctness               0.347728
5   answer_similarity               0.907992
6                mean               0.398745


In [39]:

df.head()

Metric,index,faithfulness,answer_relevancy,context_recall,context_precision,answer_correctness,answer_similarity,mean
0,setting6_sq_retriever,0.258532,0.553217,0.1,0.225,0.347728,0.907992,0.398745
